# 3 Extract facial data from videos.

We use a range of libraries to extract facial data from the videos. The main library is [DeepFace](https://github.com/serengil/deepface) but we also try FER - [Facial Expression Recognition](https://github.com/justinshenk/fer).

`pip install deepface`



## 3.1 DeepFace



In [ ]:
from deepface import DeepFace
import pandas as pd
import pprint
import cv2
import os
import time
import utils


In [ ]:
videos_in = r"..\LookitLaughter.test"
demo_data = r"..\data\demo"
data_dir = r"..\data\1_interim"


test_video = "2UWdXP.joke1.rep2.take1.Peekaboo.mp4"
test_video = "6c6MZQ.joke1.rep1.take1.ThatsNotAHat.mp4"

test_image = "peekaboo.png"
test_image2 = "mother-and-baby.jpg"

In [ ]:
#just playing around to try out different models built into deepface
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
]

result = DeepFace.analyze(img_path=os.path.join(demo_data,test_image2), enforce_detection = False, detector_backend = backends[1])

for r in result:
    pprint.pprint(r)


In [ ]:
def getfacedataforvideo(video_path, backend = 'ssd'):
    print("Processing video: ", video_path)
    cap = cv2.VideoCapture(video_path)
    success, image = cap.read()
    if not success:
        print("Failed to read video: ", video_path)
        return None
    facesdf = utils.createfacesdf()
    frameidx = 0
    fails = []
    while success:
        try:  
            faces = DeepFace.analyze(img_path = image, 
                                    enforce_detection = True, 
                                    actions = ('age','gender','emotion'),
                                    detector_backend = backend)
        except:
            #print(f"deepface.analyse failed for frame {frame}" )
            fails.append(frameidx)
            faces = []
        if len(faces) > 0:
            facesdf = utils.addfacestodf(facesdf,frameidx,faces)
        success,image = cap.read()
        frameidx += 1
    
    print(f"Failed to process {len(fails)}/{frameidx} ({(round(100*len(fails)/frameidx, 1))}%) frames")
    cap.release()
    return facesdf

In [ ]:
processedvideos = utils.getprocessedvideos(data_dir)
processedvideos.head()

In [ ]:
forceFaces = False
backend = "ssd"

for index, r in processedvideos.iterrows():
    if forceFaces or pd.isnull(r["Faces.file"]) or not os.path.exists(r["Faces.file"]):
        filepath = os.path.join(videos_in, r["VideoID"])
        facesdf = getfacedataforvideo(filepath, backend = backend)
        if not facesdf is None:
            stemname = os.path.splitext(r["VideoID"])[0]
            facefile =  os.path.join(data_dir, stemname + f".faces.{backend}.csv")
            facesdf.to_csv(facefile)
            r["Faces.file"] = facefile
            r["Faces.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        else:
            r["Faces.file"] = ""
            r["Faces.when"] = ""
    #update this row in processedvideos dataframe
    processedvideos.loc[index] = r
utils.saveprocessedvideos(processedvideos, data_dir)

In [7]:
processedvideos.head()

,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,FPS,...,Audio.file,Faces.when,Faces.file,LastError,Speech.file,Speech.when,Objects.file,Objects.when,Understand.file,Understand.when
Index,,,,,,,,,,,,,,,,,,,,,
0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,217,14.298910,...,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-10-03 13:10:37,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38,NaN,NaN,NaN,NaN
1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,152,14.359089,...,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-10-03 13:12:51,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39,NaN,NaN,NaN,NaN
2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,95,13.241315,...,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-10-03 13:14:15,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40,NaN,NaN,NaN,NaN
3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,97,14.213813,...,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-10-03 13:15:18,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40,NaN,NaN,NaN,NaN
4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,133,14.223092,...,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-10-03 13:17:30,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48,NaN,NaN,NaN,NaN


In [ ]:
#TODO - add code to match the person labels for the faces to person labels from pose detection (step 1)
